# 3.7 softmax的pytorch实现

In [15]:
import torch
import torch.nn as nn
from torch.utils import data
from torchvision import datasets, transforms
import torch.optim

### load the data and param

In [16]:
BATCH_SIZE = 32
EPOCH = 10
INPUT = 28* 28
OUTPUT = 10
LR = 0.001
transform = transforms.Compose([transforms.ToTensor()])
train_dataset = datasets.FashionMNIST(root = './data', train=True, download=True, transform=transform)
test_dataset = datasets.FashionMNIST(root = './data', train=False, download=True, transform=transform)

train_iter = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_iter = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

### create the net

In [ ]:

class softmax_Net(nn.Module):
    def __init__(self, input, output):
        super().__init__()
        self.net = nn.Sequential(nn.Flatten(), nn.Linear(input, output))
        
    def forward(self, x):
        return self.net(x)

net = softmax_Net(INPUT, OUTPUT)


### train

In [ ]:
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=LR)


for i in range(EPOCH):
    for x, y in train_iter:
        y = torch.tensor()
        print(x.shape, y.shape)
        net.zero_grad()
        o = net(x)
        print(o.shape)
        l = loss(y - o)
        loss.backward()
        optimizer.step()

torch.Size([32, 1, 28, 28]) torch.Size([32])
torch.Size([32, 10])


e:\Anaconda3\envs\pytorch\lib\site-packages\torch\nn\modules\module.py:1532: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


RuntimeError: The size of tensor a (32) must match the size of tensor b (10) at non-singleton dimension 1